In [1]:
import sys
import torch
import numpy as np
from tqdm import tqdm
from torch.autograd import Variable
sys.path.append('../../model/geometric_temporal/')
sys.path.append('../../dataset/')

In [2]:
from recurrent import LSTMGCNModel
from gdelt_dataset import GDELTDatasetLoader
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal
seed = torch.random.manual_seed(52)

In [3]:
dtype = torch.float
device = torch.device("cpu")

In [4]:
loader = GDELTDatasetLoader('../../data/dataset/gdelt_data_2.pkl', 1000)
dataset = loader.get_dataset()

In [5]:
model = LSTMGCNModel(7, 5, 1, 3)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)

In [6]:
list(model.parameters())

[Parameter containing:
 tensor([[ 2.6046e-01,  2.3336e-01, -4.0358e-01, -5.4300e-02, -3.7106e-01],
         [ 6.7656e-01, -6.8942e-01, -3.4608e-01,  3.3522e-02,  4.6748e-01],
         [ 5.1704e-01, -4.8018e-01, -2.1699e-01,  2.9059e-01,  1.8975e-01],
         [ 4.2044e-01,  3.2515e-01, -6.5118e-01, -2.1279e-01,  7.0990e-03],
         [-1.3022e-01,  3.5641e-01, -4.8853e-01,  4.0386e-01,  1.7466e-01],
         [ 3.2112e-01,  1.6990e-01, -6.6140e-01,  3.8928e-04,  1.2960e-01],
         [ 5.0093e-01, -5.8758e-01, -1.2861e-01, -3.2961e-01, -1.2037e-01]],
        requires_grad=True),
 Parameter containing:
 tensor([[0., 0., 0., 0., 0.]], requires_grad=True),
 Parameter containing:
 tensor([[ 0.3509,  0.2890,  0.1128, -0.0027,  0.4863],
         [ 0.3039,  0.4165,  0.5185, -0.2399,  0.5408],
         [ 0.0609, -0.2603,  0.5564, -0.5055,  0.0474],
         [ 0.1932,  0.2182,  0.2020,  0.1924,  0.5180],
         [ 0.5329, -0.0621, -0.4194,  0.4961,  0.3963],
         [ 0.1215, -0.5707,  0.2380,

In [7]:
model.train()

for epoch in tqdm(range(10)):
    cost = 0
        
    for time, snapshot in enumerate(dataset):
        max_x = torch.nan_to_num(snapshot.x).max()
        X = torch.div(snapshot.x, max_x + 1)

        max_w = torch.nan_to_num(snapshot.edge_attr).max()
        W = torch.div(snapshot.edge_attr, max_w + 1)

        y_hat = model(X, snapshot.edge_index, W)

        inner_cost = 0
        for i, j in zip(snapshot.edge_index[0], snapshot.edge_index[1]):
            pred = torch.dot(y_hat[i], y_hat[j])
            if torch.isnan(pred):
                pred = 0
            true_val = snapshot.y[i][j]
            if torch.isnan(true_val):
                true_val = 0
            exp = torch.pow(pred-true_val, 2)
            inner_cost += exp
        inner_cost /= (snapshot.edge_index.shape[0])
        cost += inner_cost
        
    cost = cost / (time+1)
    print(f'Cost: {cost}, Epoch: {epoch}')
    cost.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 2, norm_type=2.0, error_if_nonfinite=False)
                                   
    for name, param in model.named_parameters():
        print(name, param.grad)
        print(name, param.data)
                                   
    optimizer.step()
    optimizer.zero_grad()

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Cost: 120.0234603881836, Epoch: 0


 10%|████████▎                                                                          | 1/10 [00:05<00:46,  5.21s/it]

lstm.W_i tensor([[nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]])
lstm.W_i tensor([[ 2.6046e-01,  2.3336e-01, -4.0358e-01, -5.4300e-02, -3.7106e-01],
        [ 6.7656e-01, -6.8942e-01, -3.4608e-01,  3.3522e-02,  4.6748e-01],
        [ 5.1704e-01, -4.8018e-01, -2.1699e-01,  2.9059e-01,  1.8975e-01],
        [ 4.2044e-01,  3.2515e-01, -6.5118e-01, -2.1279e-01,  7.0990e-03],
        [-1.3022e-01,  3.5641e-01, -4.8853e-01,  4.0386e-01,  1.7466e-01],
        [ 3.2112e-01,  1.6990e-01, -6.6140e-01,  3.8928e-04,  1.2960e-01],
        [ 5.0093e-01, -5.8758e-01, -1.2861e-01, -3.2961e-01, -1.2037e-01]])
lstm.b_i tensor([[nan, nan, nan, nan, nan]])
lstm.b_i tensor([[0., 0., 0., 0., 0.]])
lstm.W_f tensor([[nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [na

 10%|████████▎                                                                          | 1/10 [00:08<01:19,  8.86s/it]

Cost: 121.29145050048828, Epoch: 1


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn